In [56]:
import pandas as pd
import unicodedata

In [63]:
def add_space_if_fw(text):
    if unicodedata.east_asian_width(text) in 'FWA':
        return text + ' '
    else:
        return text

In [65]:
df = {}
for i in range(1,3):
    file_name = f'test{i}.csv'
    with open(file_name, 'r', encoding='UTF-8') as f:
        content = f.read()
    result = ''
    for ch in content:
        result += add_space_if_fw(ch)
    file_name_changed = f'changed_test{i}.csv'
    with open(file_name_changed, mode='w', encoding='UTF-8') as f:
        f.write(result)
    widths = [7, 6]
    if i == 1:
        names = ['name', 'code']
    else:
        names = ['code', 'text']
    df[i] = pd.read_fwf(file_name_changed, widths=widths, names=names, encoding='UTF-8')
for i in df:
    print(df[i])

  name  code
0  abc   123
1  efg   456
2  hij   789
   code text
0   123  xxx
1   123  yyy
2   456  zzz


In [69]:
import sqlite3

In [84]:
con = sqlite3.connect('test.db')
cur = con.cursor()

In [87]:
# テーブル削除
cur.execute(
    'DROP TABLE IF EXISTS items'
)

In [89]:
# 空のテーブル作成
cur.execute(
    'CREATE TABLE items1 (id INTEGER PRIMARY KEY, name TEXT, code INTEGER)'
)
cur.execute(
    'CREATE TABLE items2 (id INTEGER PRIMARY KEY, code INTEGER, text TEXT)'
)

In [90]:
# データフレームからテーブル作成
for i in df:
    df[i].to_sql(f'items{i}', con, if_exists='append', index = False)

    cur.execute(
        f'SELECT * FROM items{i}'
    )
    print(cur.fetchall())

[(1, 'abc', 123), (2, 'efg', 456), (3, 'hij', 789)]
[(1, 123, 'xxx'), (2, 123, 'yyy'), (3, 456, 'zzz')]


In [91]:
# 全テーブル検索
cur.execute(
    "SELECT NAME FROM sqlite_master WHERE type='table'"
)
tables = cur.fetchall()
tables

[('sqlite_sequence',), ('items1',), ('items2',)]

In [92]:
con.close()

In [151]:
# データベース操作
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = 'sqlite:///test.db'


engine = create_engine(
  SQLALCHEMY_DATABASE_URL,
  connect_args={'check_same_thread': False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()



In [152]:
# モデル設定
from sqlalchemy import Column, ForeignKey, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

class Item(Base):
  __tablename__ = 'items1'
  id = Column(Integer, primary_key=True, autoincrement=True, index=True)
  name = Column(String, unique=True, index=True)
  code = Column(Integer, index=True)
  item2 = relationship('Item2')

class Item2(Base):
  __tablename__ = 'items2'
  id = Column(Integer, primary_key=True, autoincrement=True, index=True)
  code = Column(Integer, ForeignKey('items1.code'), index=True)
  text = Column(String, unique=True, index=True)

In [158]:
session = SessionLocal()
item = session.query(Item).first()
for i in item.item2:
  print(i.text)

xxx
yyy
